In [11]:
# -----------------------------
# 1) HfFolder Monkey Patch (huggingface_hub 1.x 대응)
# -----------------------------
try:
    from huggingface_hub import HfFolder
except ImportError:
    # huggingface_hub>=1.0.0 에서 HfFolder가 제거되었기 때문에 직접 구현
    from huggingface_hub import get_token, login, logout
    import sys
    import types

    class HfFolder:
        @staticmethod
        def get_token():
            try:
                return get_token()
            except Exception:
                return None

        @staticmethod
        def save_token(token: str):
            login(token=token)

        @staticmethod
        def delete_token():
            logout()

    # huggingface_hub 모듈에 강제로 삽입
    module = sys.modules.get("huggingface_hub", types.ModuleType("huggingface_hub"))
    module.HfFolder = HfFolder
    sys.modules["huggingface_hub"] = module


In [15]:
#
# simple version >> stable
#
import gradio as gr

def chat_response(message, history):
    """LangChain 챗봇 응답"""
    if not message.strip():
        return "질문을 입력해주세요."

    try:
        response = myChain.invoke({"query": message})
        return response["result"]
    except Exception as e:
        return f"❌ 오류: {str(e)}"

# 가장 간단한 인터페이스
demo = gr.ChatInterface(
    fn=chat_response,
    type="messages",
    title="📚 경제금융용어 챗봇",
    examples=["GDP란?", "인플레이션이란?"]
)

demo.launch()

Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


In [2]:
#
# gr.ChatInterface() 테스트
#
import gradio as gr

def respond(message, history):
    return f"Echo: {message}"

demo = gr.ChatInterface(respond)
demo.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 3.50.2, however version 4.44.1 is available, please upgrade.
--------


In [3]:
#
# gr.Blocks() 테스트
#
import gradio as gr

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Ask me")
    submit = gr.Button("Send")

    def respond(message, history):
        return "", history + [(message, "Hello!")]

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    submit.click(respond, [msg, chatbot], [msg, chatbot])

demo.launch()


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 3.50.2, however version 4.44.1 is available, please upgrade.
--------


In [22]:
import gradio as gr
import random
import time

def respond(message, history):
    # history는 gr.State로 초기화된 리스트
    bot_message = random.choice(["How are you?", "Today is a great day", "I'm very hungry"])
    # 안전하게 새로운 리스트를 만들어 반환 (불변 패턴)
    new_history = history + [(message, bot_message)]
    # 반환값 순서에 주의: outputs = [textbox, chatbot, state] 이므로
    # 첫값은 textbox의 새값 (""), 두번째는 chatbot에 표시할 history, 세번째는 state에 저장할 history
    return "", new_history, new_history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Emma Chatbot")
    msg = gr.Textbox(placeholder="Type your message and press Enter")
    submit = gr.Button("Send")
    clear = gr.Button("Clear")

    # 상태 저장용 State 컴포넌트 (반드시 초기값으로 리스트 제공)
    state = gr.State([])

    # Textbox에서 Enter 누르면 respond 호출
    # inputs: [msg, state]
    # outputs: [msg (Textbox), chatbot (Chatbot), state (State)]
    msg.submit(respond, inputs=[msg, state], outputs=[msg, chatbot, state])

    # 버튼 클릭도 동일하게 연결
    submit.click(respond, inputs=[msg, state], outputs=[msg, chatbot, state])

    # Clear 버튼: textbox는 빈 문자열로, chatbot과 state는 빈 리스트로 초기화
    def do_clear():
        return "", [], []
    clear.click(do_clear, inputs=None, outputs=[msg, chatbot, state])

demo.launch(server_name="127.0.0.1", server_port=7851, debug=True, share=True)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/Users/doogie/opt/anaconda3/envs/py09/lib/python3.9/site-packages/uvicorn/protocols/http/httptools_impl.py", line 409, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/Users/doogie/opt/anaconda3/envs/py09/lib/python3.9/site-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
  File "/Users/doogie/opt/anaconda3/envs/py09/lib/python3.9/site-packages/fastapi/applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "/Users/doogie/opt/anaconda3/envs/py09/lib/python3.9/site-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/Users/doogie/opt/anaconda3/envs/py09/lib/python3.9/site-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/Users/doogie/opt/anaconda3/envs/py09/lib/py

Running on local URL:  http://127.0.0.1:7851


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/Users/doogie/opt/anaconda3/envs/py09/lib/python3.9/site-packages/uvicorn/protocols/http/httptools_impl.py", line 409, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/Users/doogie/opt/anaconda3/envs/py09/lib/python3.9/site-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
  File "/Users/doogie/opt/anaconda3/envs/py09/lib/python3.9/site-packages/fastapi/applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "/Users/doogie/opt/anaconda3/envs/py09/lib/python3.9/site-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/Users/doogie/opt/anaconda3/envs/py09/lib/python3.9/site-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/Users/doogie/opt/anaconda3/envs/py09/lib/py

Running on public URL: https://862a0f78b0f9552f5e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/Users/doogie/opt/anaconda3/envs/py09/lib/python3.9/site-packages/uvicorn/protocols/http/httptools_impl.py", line 409, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/Users/doogie/opt/anaconda3/envs/py09/lib/python3.9/site-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
  File "/Users/doogie/opt/anaconda3/envs/py09/lib/python3.9/site-packages/fastapi/applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "/Users/doogie/opt/anaconda3/envs/py09/lib/python3.9/site-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/Users/doogie/opt/anaconda3/envs/py09/lib/python3.9/site-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/Users/doogie/opt/anaconda3/envs/py09/lib/py

KeyboardInterrupt: 